# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

### 데이터 셋 읽어오기

In [2]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [3]:
df_train.head() # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


In [4]:
df_train.isna().sum()

bant_submit                    0
customer_country             982
business_unit                  0
com_reg_ver_win_rate       44731
customer_idx                   0
customer_type              43961
enterprise                     0
historical_existing_cnt    45543
id_strategic_ver           55855
it_strategic_ver           58178
idit_strategic_ver         54734
customer_job               18733
lead_desc_length               0
inquiry_type                 941
product_category           19374
product_subcategory        50064
product_modelname          50070
customer_country.1           982
customer_position              0
response_corporate             0
expected_timeline          30863
ver_cus                        0
ver_pro                        0
ver_win_rate_x             40882
ver_win_ratio_per_bu       43995
business_area              40882
business_subarea           53773
lead_owner                     0
is_converted                   0
dtype: int64

### 결측치 제거

In [5]:
df = df_train.copy()

In [6]:
"""# 결측치를 각 수치형 컬럼의 평균값으로 채우기 (for 루프 사용)
numeric_columns = df_train.select_dtypes(include=['number']).columns
for column in numeric_columns:
    df_train[column].fillna(df_train[column].mean(), inplace=True)"""

"# 결측치를 각 수치형 컬럼의 평균값으로 채우기 (for 루프 사용)\nnumeric_columns = df_train.select_dtypes(include=['number']).columns\nfor column in numeric_columns:\n    df_train[column].fillna(df_train[column].mean(), inplace=True)"

In [7]:
#'historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver' 결측치 0 변환
columns_to_replace = ['historical_existing_cnt', 'id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver']
df[columns_to_replace] = df[columns_to_replace].fillna(0)

In [8]:
# 0처리 or 중앙값 적용. 현재 0적용
columns_to_replace = ['ver_win_rate_x', 'ver_win_ratio_per_bu']
df[columns_to_replace] = df[columns_to_replace].fillna(0)

In [9]:
"""# 결측치를 각 범주형 컬럼의 최빈값으로 채우기 (for 루프 사용)
categorical_columns = df_train.select_dtypes(include=['object']).columns
for column in categorical_columns:
    df_train[column].fillna(df_train[column].mode()[0], inplace=True)"""

"# 결측치를 각 범주형 컬럼의 최빈값으로 채우기 (for 루프 사용)\ncategorical_columns = df_train.select_dtypes(include=['object']).columns\nfor column in categorical_columns:\n    df_train[column].fillna(df_train[column].mode()[0], inplace=True)"

In [10]:
# 이상치가 44731개라 중앙값으로 결측치 대체
median_value = df['com_reg_ver_win_rate'].median()
print(median_value)
df['com_reg_ver_win_rate'] = df['com_reg_ver_win_rate'].fillna(median_value)

0.0491803278688524


In [11]:
# 지정된 컬럼들에 대해서만 최빈값으로 NaN을 치환합니다.
columns_to_replace_mode = [
    'customer_country', 'customer_type', 'customer_job', 'inquiry_type', 
    'product_category', 'product_subcategory', 'product_modelname', 
    'expected_timeline', 'business_area', 'business_subarea'
]

for column in columns_to_replace_mode:
    # 컬럼별 최빈값을 계산합니다.
    mode_value = df[column].mode()[0]
    
    # NaN 값을 최빈값으로 치환합니다.
    df[column] = df[column].fillna(mode_value)

In [12]:
df['com_reg_ver_win_rate']

0        0.066667
1        0.066667
2        0.088889
3        0.088889
4        0.088889
           ...   
59294    0.049180
59295    0.040000
59296    0.040000
59297    0.040000
59298    0.049180
Name: com_reg_ver_win_rate, Length: 59299, dtype: float64

In [13]:
# customer_country.1 컬럼에 대해 국가만 남기고 나머지 정보 생략
df['customer_country.1'] = df['customer_country.1'].str.split('/').str[-1].str.strip()

In [14]:
# 국가명에 공백이 있는경우 해당 공백 제거
df['customer_country.1'] = df['customer_country.1'].str.replace(' ', '')

In [15]:
df['customer_country.1']

0        Philippines
1        Philippines
2              India
3              India
4              India
            ...     
59294         Poland
59295       Colombia
59296           Peru
59297           Peru
59298         France
Name: customer_country.1, Length: 59299, dtype: object

In [16]:
mode_value = df['customer_country.1'].mode()[0]
df['customer_country.1'] = df['customer_country.1'].fillna(mode_value)

In [17]:
df.isnull().sum()

bant_submit                0
customer_country           0
business_unit              0
com_reg_ver_win_rate       0
customer_idx               0
customer_type              0
enterprise                 0
historical_existing_cnt    0
id_strategic_ver           0
it_strategic_ver           0
idit_strategic_ver         0
customer_job               0
lead_desc_length           0
inquiry_type               0
product_category           0
product_subcategory        0
product_modelname          0
customer_country.1         0
customer_position          0
response_corporate         0
expected_timeline          0
ver_cus                    0
ver_pro                    0
ver_win_rate_x             0
ver_win_ratio_per_bu       0
business_area              0
business_subarea           0
lead_owner                 0
is_converted               0
dtype: int64

In [18]:
df['lead_desc_length'].value_counts()

lead_desc_length
3      10378
14      3180
2        934
10       891
4        884
       ...  
824        1
559        1
676        1
907        1
726        1
Name: count, Length: 909, dtype: int64

In [19]:
df['lead_desc_length']

0         62
1         96
2         56
3         44
4         97
        ... 
59294    200
59295     70
59296     34
59297    377
59298     78
Name: lead_desc_length, Length: 59299, dtype: int64

### 특성 합성곱

In [20]:
df

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.00,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,0.0,0.0,0.0,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.00,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,0.0,0.0,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.00,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,0.0,0.0,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.00,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,0.0,0.0,0.0,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.00,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,0.0,0.0,0.0,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,Others,4,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59294,1.00,/Sląskie/Poland,AS,0.049180,33747,End Customer,SMB,0.0,0.0,0.0,...,LGEPL,3 months ~ 6 months,0,0,0.000026,0.028777,public facility,Others,694,False
59295,0.75,/Bogotá DC /Colombia,AS,0.040000,35420,Specifier/ Influencer,Enterprise,0.0,0.0,0.0,...,LGECB,9 months ~ 1 year,0,0,0.000026,0.028777,public facility,Others,39,False
59296,0.75,/Pisco/Peru,AS,0.040000,19249,Specifier/ Influencer,Enterprise,0.0,0.0,0.0,...,LGEPR,less than 3 months,0,0,0.000026,0.028777,public facility,Others,125,False
59297,1.00,/santa cruz bolivia/Peru,AS,0.040000,40327,End-Customer,Enterprise,0.0,0.0,0.0,...,LGEPR,more than a year,0,0,0.000026,0.028777,public facility,Others,134,False


### 레이블 인코딩

In [21]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# 모든 컬럼에 대해 레이블 인코딩 진행
for column in df.columns:
    df[column] = le.fit_transform(df[column])

In [22]:
df.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,4,7546,0,40,23743,10,0,0,0,0,...,33,246,1,0,12,5,0,28,0,1
1,4,6887,0,40,17000,10,0,12,0,0,...,33,246,1,0,12,5,0,0,1,1
2,4,5037,0,52,1282,10,0,107,0,0,...,21,246,1,0,12,5,0,17,2,1
3,4,1943,0,52,3571,10,0,0,0,0,...,21,246,1,0,12,5,0,44,3,1
4,4,4307,0,52,12474,29,0,0,0,0,...,21,246,0,0,12,5,0,62,4,1


학습 데이터와 제출 데이터 분리

In [23]:
for col in df.columns:  
    df_train[col] = df.iloc[: len(df_train)][col]
    df_test[col] = df.iloc[len(df_train) :][col]

### 학습, 검증 데이터 분리

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

### DecisionTreeClassifer

In [25]:
"""from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# DecisionTreeClassifier의 주요 하이퍼파라미터들을 설정
param_grid = {
    'max_depth': [10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['gini', 'entropy']
}

# DecisionTreeClassifier 모델 생성
dtc = DecisionTreeClassifier()

# GridSearchCV를 이용하여 하이퍼파라미터 튜닝
grid_search = GridSearchCV(estimator=dtc, param_grid=param_grid, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print('Best parameters:', grid_search.best_params_)

# 최적의 하이퍼파라미터를 가진 모델을 사용하여 예측
y_pred = grid_search.predict(X_test)"""

KeyboardInterrupt: 

In [ ]:
"""# 최적의 하이퍼파라미터를 사용하여 모델 생성
best_dtc = DecisionTreeClassifier(criterion='entropy', max_depth=30, 
                                  min_samples_leaf=2, min_samples_split=2)

# 모델 학습
best_dtc.fit(X_train, y_train)"""

In [ ]:
"""from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("정확도: {:.4f}".format(accuracy))
print("정밀도: {:.4f}".format(precision))
print("재현율: {:.4f}".format(recall))
print("F1: {:.4f}".format(f1))"""

### RandomForest

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf = RandomForestClassifier()

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

In [28]:
rf.fit(X_train, y_train)

RandomForestClassifier()

In [29]:
rf.score(X_test, y_test)

0.972849915682968

In [30]:
f1 = f1_score(y_test, y_test)
f1

1.0

### XGBoost, GridSearchCV

In [31]:
from xgboost import XGBClassifier

model =XGBClassifier()

In [32]:
from sklearn.model_selection import GridSearchCV

model_param_grid={
    'n_estimators': [2, 5, 10, 50, 100, 250, 500],
    'max_depth': [2, 4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.5, 1.0, 1.5],
    'objective': ['binary:logistic']
}

In [33]:
model_grid=GridSearchCV(model, param_grid=model_param_grid, scoring="f1_macro", n_jobs=-1, verbose=2)
model_grid.fit(X_train, y_train)

Fitting 5 folds for each of 210 candidates, totalling 1050 fits
[CV] END learning_rate=0.01, max_depth=2, n_estimators=2, objective=binary:logistic; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=5, objective=binary:logistic; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=5, objective=binary:logistic; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=10, objective=binary:logistic; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=50, objective=binary:logistic; total time=   0.4s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=100, objective=binary:logistic; total time=   0.5s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=250, objective=binary:logistic; total time=   1.1s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=250, objective=binary:logistic; total time=   1.1s
[CV] END learning_rate=0.01, max_depth=2, n_estimators=500, objective=binary:logistic; total tim

GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.05, 0.1, 0.5, 1.0, 1.5],
                         'max_depth': [2, 4, 6, 8, 10],
                         'n_estimators': [2, 5, 10, 50, 100, 250, 500],
                         'objective': ['binary:logistic']},
             scoring='f1_macro', verbose=2)

In [34]:
print(model_grid.best_score_)
print(model_grid.best_params_)

0.9033455697525767
{'learning_rate': 0.1, 'max_depth': 8, 'n_estimators': 500, 'objective': 'binary:logistic'}


In [35]:
xg_model=XGBClassifier(
    n_estimators= 500,
    max_depth=8,
    learning_rate=0.1,
    objective='binary:logistic'
)

In [36]:
xg_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [37]:
y_predict=xg_model.predict(X_test)
y_predict.sum()/len(y_predict)

0.0657672849915683

In [38]:
xg_model.score(X_test, y_test)

0.9752951096121416

In [49]:
xg_f1 = f1_score(y_test, y_predict)
xg_f1

0.8303416328894035

### 제출하기

테스트 데이터 예측

In [72]:
# 예측에 필요한 데이터 분리
X_test = df_test.drop(["is_converted", "id"], axis=1)

In [55]:
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [56]:
len(x_test)

5271

In [57]:
y_predict = xg_model.predict(X_test.fillna(0))
sum(y_predict)

780

### 제출 파일 작성

In [51]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = y_predict

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

ValueError: Length of values (11860) does not match length of index (5271)

In [61]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")

# y_predict의 길이를 df_sub의 길이에 맞게 잘라서 사용
y_predict_trimmed = y_predict[:len(df_sub)]
df_sub["is_converted"] = y_predict_trimmed

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)